# Programación declarativa @ URJC
# Programación funcional
## Examen Convocatoria Ordinaria (3 de febrero de 2021)
## Curso 20-21

# Definiciones auxiliares

In [2]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

### Algunas funciones sobre tipos estándar de la librería de Scala

In [2]:
object Signatures{
    abstract class List[A]{
        
        // Common HOFs
        def foldRight[B](directSol: B)(composeSol: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
        
        // Reverse a list
        // e.g. List(1,2,3).reverse==List(3,2,1)
        def reverse: List[A]
        
        // Take the first `n` elements of the list
        // e.g. List(1,2,3).take(2) == List(1,2)
        //      List(1,2,3).take(0) == List()
        //      List(1,2,3).take(5) == List(1,2,3)
        def take(n: Int): List[A]
        
        // Drop the first `n` elements of the list 
        // e.g. List(1,2,3).drop(2) == List(3)
        //      List(1,2,3).drop(0) == List(1,2,3)
        //      List(1,2,3).drop(4) == List()
        def drop(n: Int): List[A]

        // List concatenation
        // e.g. List(1,2,3).concat(List(4,5)) == List(1,2,3,4,5)
        def concat(l: List[A]): List[A]
    }
    
    abstract class Option[A]{
        // Test whether the value is defined (i.e. `Some`) or not (i.e. `None`)
        def isDefined: Boolean 
        def map[B](f: A => B): Option[B]
    }
    
    abstract class Either[A, B]{
        // Test whether the value is left or right
        def isLeft: Boolean 
        def isRight: Boolean 
        def map[C](f: B => C): Either[A, C]
    }
}

defined object Signatures

### Definiciones auxiliares sobre la correspondencia Curry-Howard

In [2]:
type Not[P] = P => Nothing
type <=>[P, Q] = (P => Q, Q => P)
type Or[P, Q] = Either[P, Q]
type And[P, Q] = (P, Q)

defined type Not
defined type <=>
defined type Or
defined type And

Proof the following theorem of classical logic assuming that the double negation law holds for proposition `P`:

$ \vdash (\neg p \rightarrow p) \rightarrow p$

In [6]:
// Not[Not[P]] = ((P => Nothing) => Nothing)
// Not[P] = P => Nothing

def proof[P](dn: Not[Not[P]] => P): (Not[P] => P) => P =
    npip => dn(np => np(npip(np)))

defined function proof

$\vdash \neg \neg \neg p \leftrightarrow \neg p$

In [13]:
def proofAux1[P]: (Not[Not[Not[P]]] => Not[P], Not[P] => Not[Not[Not[P]]]) =
    ???

def proofAux2[P]: ((((P => Nothing) => Nothing) => Nothing) => (P => Nothing), 
                   (P => Nothing) => (((P => Nothing) => Nothing) => Nothing)) =
    ???

// np -> P => Nothing
def proof[P]: Not[Not[Not[P]]] <=> Not[P] =
    (nnnp => (p => nnnp(np => np(p))), np => (nnp => nnp(np)))

defined function proofAux1
defined function proofAux2
defined function proof

# Ejercicio 1 (variante 1)


__a) (2 puntos)__ Utiliza la correspondencia de Curry-Howard para demostrar que los siguientes razonamientos de la lógica proposicional representan deducciones válidas de la lógica intuicionista: 

* Dilema constructivo complejo: $\{p ∨ q, p→r, q→s\} ⊢ r ∨ s$

In [17]:
def proofAux[P, Q, R, S]: Or[P, Q] => (P => R) => (Q => S) => Or[R, S] =
    ???

def proof[P, Q, R, S](porq: Or[P, Q], pir: P => R, qis: Q => S): Or[R, S] = // Or[R, S] = Either[R, S]
    porq match {
        case Left(p) =>
            Left(pir(p))
        case Right(q) =>
            Right(qis(q))
    }

defined function proofAux
defined function proof

* Dilema destructivo simple: $\{¬p ∨¬q, r→p, r→q\} ⊢ ¬r$

In [21]:
// Or[Not[P], Not[Q]] = Either[P => Nothing, Q => Nothing]
// Not[R] = R => Nothing

def proof[P, Q, R](npornq: Or[Not[P], Not[Q]], rip: R => P, riq: R => Q): Not[R] =
    r => npornq match {
        case Left(np) => // r | np | rip | riq
            np(rip(r))
        case Right(nq) => // r | nq | rip | riq
            nq(riq(r))
    }

defined function proof

* Dilema destructivo complejo: $\{¬p ∨¬q, r→p, s→q\} ⊢ ¬r ∨ ¬s$

In [27]:
// Or[Not[P], Not[Q]] = Either[P => Nothing, Q => Nothing]

// Or[Not[R], Not[S]] = Either[R => Nothing, S => Nothing]

def proof[P, Q, R, S](npornq: Or[Not[P], Not[Q]], rip: R => P, siq: S => Q): Or[Not[R], Not[S]] =
    npornq match {
        case Left(np) => // np | rip | siq
            Left(r => np(rip(r)))
        case Right(nq) =>
            Right(s => nq(siq(s)))
    }

defined function proof

__b) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar el siguiente teorema de la lógica clásica: 

$⊢ ((p →q) →p)→p$

Supóngase para ello que la ley del tercio excluso se cumple para la variable proposicional $p$, es decir, que la fórmula $p ∨ ¬p$  puede utilizarse como premisa.


In [5]:
def proof[P, Q](pornp: Or[P, Not[P]]): ((P => Q) => P) => P =
    f => pornp match {
        case Left(p) => // f | p
            p
        case Right(np) => // f: (P => Q) => P | np: P => Nothing
            f(p => np(p))
    }

defined function proof

# Ejercicio 1 (variante 2)


__a) (2 puntos)__ Utiliza la correspondencia de Curry-Howard para demostrar que los siguientes teoremas de la lógica intuicionista: 

* Dilema constructivo complejo: $⊢ (p ∨ q) → (p→r) → (q→s) → (r ∨ s)$

In [7]:
def proofAux[P, Q, R, S](porq: Or[P, Q], pir: P => R, qis: Q => S): Or[R, S] = // Or[R, S] = Either[R, S]
    ???

def proof[P, Q, R, S]: Or[P, Q] => (P => R) => (Q => S) => Or[R, S] =
    porq => pir => qis => porq match {
        case Left(p) =>
            Left(pir(p))
        case Right(q) =>
            Right(qis(q))
    }

defined function proofAux
defined function proof

* Dilema destructivo simple: $⊢ (¬p ∨¬q)→(r→p)→(r→q)→¬r$

In [9]:
def proofAux[P, Q, R](npornq: Or[Not[P], Not[Q]], rip: R => P, riq: R => Q): Not[R] =
    r => npornq match {
        case Left(np) => // r | np | rip | riq
            np(rip(r))
        case Right(nq) => // r | nq | rip | riq
            nq(riq(r))
    }

def proof[P, Q, R]: Or[Not[P], Not[Q]] => (R => P) => (R => Q) => Not[R] =
    npornq => rip => riq => (r => npornq match {
        case Left(np) => // r | np | rip | riq
            np(rip(r))
        case Right(nq) => // r | nq | rip | riq
            nq(riq(r))
    })

defined function proofAux
defined function proof

* Dilema destructivo complejo: $⊢(¬p ∨¬q)→(r→p)→(s→q) → (¬r ∨ ¬s)$

In [11]:
def proofAux[P, Q, R, S](npornq: Or[Not[P], Not[Q]], rip: R => P, siq: S => Q): Or[Not[R], Not[S]] =
    npornq match {
        case Left(np) => // np | rip | siq
            Left(r => np(rip(r)))
        case Right(nq) =>
            Right(s => nq(siq(s)))
    }

def proof[P, Q, R, S]: Or[Not[P], Not[Q]] => (R => P) => (S => Q) => Or[Not[R], Not[S]] =
    npornq => rip => siq => npornq match {
        case Left(np) => // np | rip | siq
            Left(r => np(rip(r)))
        case Right(nq) =>
            Right(s => nq(siq(s)))
    }

defined function proofAux
defined function proof

__b) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar el siguiente teorema de la lógica clásica: 

$⊢ ((p →q) →p)→p$

Supóngase para ello que la ley de la doble negación se cumple para la variable proposicional $p$, es decir, que la fórmula  $¬¬p→p$  puede utilizarse como premisa.

In [4]:
// Not[Not[P]] = ((P => Nothing) => Nothing)

def proof[P, Q](dn: Not[Not[P]] => P): ((P => Q) => P) => P =
    f => dn(np => np(f(np)))

defined function proof

# Ejercicio 2 
__(1 punto)__

Demuestra el siguiente isomorfismo entre tipos algebraicos de datos para todo tipo $X$: 

$(X+1)^2 \cong X^2+2*X+1$

de tal forma que se verifique el siguiente test unitario para $X=Boolean$:

_(variante 1)_

In [17]:
class IsoTest(
    from: ((Option[Boolean], Option[Boolean])) => Either[(Boolean, Boolean), Either[Boolean, Option[Boolean]]],
    to: Either[(Boolean, Boolean), Either[Boolean, Option[Boolean]]] => (Option[Boolean], Option[Boolean])
) extends FlatSpec with Matchers {
    "from-to" should "work" in {
        from(to(Left((true,true)))) shouldBe Left((true, true))
        from(to(Right(Left(true)))) shouldBe Right(Left(true))
        from(to(Right(Right(Some(true))))) shouldBe Right(Right(Some(true)))
        from(to(Right(Right(None)))) shouldBe Right(Right(None))
    }
    
    "to-from" should "work" in {
        to(from((None, None))) shouldBe (None, None)
        to(from((Some(false), None))) shouldBe (Some(false), None)
        to(from((None, Some(true)))) shouldBe (None, Some(true))
        to(from((Some(true), Some(false)))) shouldBe (Some(true), Some(false))
    }
}

defined class IsoTest

In [18]:
// Option: None = 1 | Some(X) = X
def from[X](l: (Option[X], Option[X])): Either[(X, X), Either[X, Option[X]]] = 
    l match {
        case (Some(x1), Some(x2)) =>
            Left((x1, x2))
        case (Some(x), None) =>
            Right(Left(x))
        case (None, Some(x)) =>
            Right(Right(Some(x)))
        case (None, None) =>
            Right(Right(None))
    }

defined function from

In [1]:
def to[X](l: Either[(X, X), Either[X, Option[X]]]): (Option[X], Option[X]) = 
    l match {
        case Left((x1, x2)) =>
            (Some(x1), Some(x2))
        case Right(Left(x)) =>
            (Some(x), None)
        case Right(Right(Some(x))) =>
            (None, Some(x))
        case Right(Right(None)) =>
            (None, None)
    }

defined function to

In [20]:
run(new IsoTest(from, to))

cmd16$Helper$IsoTest:
from-to
- should work
to-from
- should work


_(variante 2)_

In [21]:
class IsoTest(
    from: ((Either[Boolean, Unit], Either[Boolean, Unit])) => 
                Either[(Boolean, Boolean), Either[Boolean, Either[Boolean, Unit]]],
    to: Either[(Boolean, Boolean), Either[Boolean, Either[Boolean, Unit]]] => 
                (Either[Boolean, Unit], Either[Boolean, Unit])
) extends FlatSpec with Matchers {
    "from-to" should "work" in {
        from(to(Left((true, true)))) shouldBe Left((true, true))
        from(to(Right(Left(true)))) shouldBe Right(Left(true))
        from(to(Right(Right(Left(true))))) shouldBe Right(Right(Left(true)))
        from(to(Right(Right(Right(()))))) shouldBe Right(Right(Right(())))
    }
    
    "to-from" should "work" in {
        to(from((Right(()), Right(())))) shouldBe (Right(()), Right(()))
        to(from((Left(false), Right(())))) shouldBe (Left(false), Right(()))
        to(from((Right(()), Left(true)))) shouldBe (Right(()), Left(true))
        to(from((Left(true), Left(false)))) shouldBe (Left(true), Left(false))
    }
}

defined class IsoTest

In [22]:
// Unit: () = 1 | X = X
def from[X](l: (Either[X, Unit], Either[X, Unit])): Either[(X, X), Either[X, Either[X, Unit]]] = 
    l match {
        case (Left(x1), Left(x2)) => // X * X
            Left((x1, x2))
        case (Left(x), Right(())) => // X * 1
            Right(Left(x))
        case (Right(()), Left(x)) => // 1 * X
            Right(Right(Left(x)))
        case (Right(()), Right(())) => // 1 * 1
            Right(Right(Right(())))
    }

defined function from

In [23]:
def to[X](l: Either[(X, X), Either[X, Either[X, Unit]]]): (Either[X, Unit], Either[X, Unit]) = 
    l match {
        case Left((x1, x2)) =>
            (Left(x1), Left(x2))
        case Right(Left(x)) =>
            (Left(x), Right(()))
        case Right(Right(Left(x))) =>
            (Right(()), Left(x))
        case Right(Right(Right(()))) =>
            (Right(()), Right(()))
    } 

defined function to

In [24]:
run(new IsoTest(from, to))

cmd20$Helper$IsoTest:
from-to
- should work
to-from
- should work


# Ejercicio 3 (variante 1)
__(3 puntos)__

La función de orden superior `sequence` recibe una lista de valores opcionales y devuelve una lista con todos los valores pertenecientes a la lista de entrada (en el mismo orden), _en caso de que todos los valores de la lista de entrada estén definidos_; si alguno de los valores opcionales de la lista de entrada es `None`, entonces la función sequence devuelve `None` también. El comportamiento de la función se ilustra en el siguiente test unitario:


In [26]:
class SequenceTest(
    sequence: List[Option[Int]] => Option[List[Int]]
) extends FlatSpec with Matchers {
    "sequence" should "work" in {
        sequence(List(Some(1), Some(2), Some(3))) shouldBe Some(List(1,2,3))
        sequence(List(None, Some(2), Some(3))) shouldBe None
        sequence(List(Some(1), None, Some(3))) shouldBe None
        sequence(List(Some(5))) shouldBe Some(List(5))
        sequence(List(None)) shouldBe None
        sequence(List()) shouldBe Some(List())
    }
}

defined class SequenceTest

__a) (1 punto)__ Implementa la función `sequence` de manera recursiva.

In [4]:
def sequence[A](list: List[Option[A]]): Option[List[A]] =
    list match {
        case List() => // case Nil =>
            Some(List())
        case head :: tail =>
            head match {
                case None => // Option[A]
                    None // Option[List[A]]
                case Some(v) =>
                    sequence(tail) match {
                        case None => // Option[List[A]]
                            None
                        case Some(r) =>
                            Some(v :: r)
                    }
            }
    }

/*
Ej.: 
1. sequence(List(Some(1), None, Some(3))) -> head = Some(1) :: tail = List(None, Some(3))
 Some(1) match {
    case None => X
        None
    case Some(1) => OK! | v = 1
        None match { ||
         None  
        }
 }
2. sequence(List(None, Some(3))) -> head = None :: tail = List(Some(3))
 None
*/

/*
1. sequence(List(Some(1), Some(3))) -> head = Some(1) :: tail = List(Some(3)) -> Some(List(1, 3))x
 Some(1) match {
    case None => X
        None
    case Some(1) => OK! | v = 1
        Some(List(3)) match { ||
         case None => X
             None
         case Some(List(3)) => OK! | r = List(3)
             Some(1 :: List(3)) -> Some(List(1, 3))  
        }
 }
2. sequence(List(Some(3))) -> head = Some(3) :: tail = List()
 Some(3) match {
    case None => X
        None
    case Some(3) => OK! | v = 3
        Some(List()) match { ||
         case None => X
             None
         case Some(List()) => OK! | r = List()
             Some(3 :: List()) -> Some(List(3))
        }
 }
3. sequence(List()) -> Some(List())
*/

defined function sequence

In [32]:
run(new SequenceTest(sequence))

cmd25$Helper$SequenceTest:
sequence
- should work


__b) (1 punto)__ Implementa la función `sequence` utilizando __`foldRight`__.

In [35]:
def sequenceFR[A](list: List[Option[A]]): Option[List[A]] =
    list.foldRight(Some(List()): Option[List[A]]){
        case (_, None) => // Si me llega por la derecha que 'acc' es None, directamente devuelvo para la siguiente iteración 'None'
            None
        case (None, _) => // Si el elemento sobre el que estamos es 'None', directamente ...
            None
        case (Some(v), Some(r)) =>
            Some(v :: r)
    }

defined function sequenceFR

In [36]:
run(new SequenceTest(sequenceFR))

cmd25$Helper$SequenceTest:
sequence
- should work


__c) (1 punto)__ Se desea implementar una función que reciba una lista de enteros, divida todos sus elementos entre un valor dado y, finalmente, multiplique los resultados de las divisiones, _siempre y cuando todas las divisiones hayan resultado en un valor entero_. En caso de que la división de algunos de los elementos no haya sido entera o no se haya podido realizar (en el caso de la división por cero), la función no devolverá ningún número. Por ejemplo:



In [3]:
class TestOp(op: (List[Int], Int) => Option[Int]) extends FlatSpec with Matchers {
    "op" should "work" in {
        op(List(2,4,6), 2) shouldBe Some(2/2*4/2*6/2)
        op(List(3,6,9), 3) shouldBe Some(6)
        op(List(3,5,9), 3) shouldBe None
        op(List(), 5) shouldBe Some(1)
        op(List(), 0) shouldBe Some(1)
        op(List(2,5,2), 0) shouldBe None
    }
}

defined class TestOp

Implementa la función `op` utilizando `sequence` y otras funciones de orden superior del catálogo explicado en clase.

In [5]:
def op(list: List[Int], n: Int): Option[Int] =
    sequence(list.map{ nList =>
        if (n != 0 && nList % n == 0) { // OK!
            Some(nList / n)
        } else {
            None
        }
    }) match {
        case None => // Option[List[Int]]
            None // Option[Int]
        case Some(l) =>
            Some(l.foldLeft(1){ (acc, elem) =>
                acc * elem
            })
    }

/*
    sequence(List(2,4,6).map{ => List(Some(1), Some(2), Some(3))
        1ª It. nList -> 2 => Some(1)
        2ª It. nList -> 4 => Some(2)
        3ª It. nList -> 6 => Some(3)
    }) => Option[List(1, 2, 3)]
    
    List(1, 2, 3).foldLeft(1) { (acc, elem) =>
        1ª It. acc = 1 | elem = 1 -> 1
        2ª It. acc = 1 | elem = 2 -> 2
        3ª It. acc = 2 | elem = 3 -> 6
    } => 6
*/

defined function op

In [6]:
run(new TestOp(op))

cmd2$Helper$TestOp:
op
- should work


# Ejercicio 3 (variante 2)
__(3 puntos)__

La función de orden superior `sequence` recibe una lista de valores de tipo `X` o `Y` y devuelve una lista con los valores de tipo `Y` pertenecientes a la lista de entrada (en el mismo orden), _en caso de que dicha lista no contenga ningún valor de tipo `X`_; en caso de que sí lo contenga, la función `sequence` devolverá el primer valor de tipo `X` encontrado. El comportamiento de la función se ilustra en el siguiente test unitario, donde la función `sequence` se encuentra particularizada para los tipos `X=String` e `Y=Int`:


In [14]:
class SequenceTest(
    sequence: List[Either[String, Int]] => Either[String, List[Int]]
) extends FlatSpec with Matchers {
    "sequence" should "work" in {
        sequence(List(Right(1), Right(2), Right(3))) shouldBe Right(List(1,2,3))
        sequence(List(Left("error1"), Right(2), Right(3))) shouldBe Left("error1")
        sequence(List(Right(1), Left("error1"), Right(3))) shouldBe Left("error1")
        sequence(List(Right(1), Left("error1"), Left("error2"))) shouldBe Left("error1")
        sequence(List(Right(5))) shouldBe Right(List(5))
        sequence(List(Left("error1"))) shouldBe Left("error1")
        sequence(List()) shouldBe Right(List())
    }
}

defined class SequenceTest

__a) (1 punto)__ Implementa la función `sequence` de manera recursiva.

In [15]:
def sequence[X, Y](list: List[Either[X, Y]]): Either[X, List[Y]] =
    list match {
        case List() =>
            Right(List())
        case head :: tail => // head: Either[X, Y]
            head match {
                case Left(x) =>
                    Left(x)
                case Right(y) =>
                    sequence(tail) match {
                        case Left(x) =>
                            Left(x)
                        case Right(ly) =>
                            Right(y :: ly)
                    }
            }
    }

/*
    1. sequence(List(Right(1), Right(2), Right(3))) => List(1, 2, 3)
        head = Right(1)
        tail = List(Right(2), Right(3))
        y = 1
        List(2, 3)
        ly = List(2, 3)
    2. sequence(List(Right(2), Right(3))) => List(2, 3)
        head = Right(2)
        tail = List(Right(3))
        y = 2
        List(3)
        ly = List(3)
    3. sequence(List(Right(3))) => List(3)
        head = Right(3)
        tail = List()
        y = 3
        Right(List())
        ly = List()
    4. sequence(List()) => Right(List())
*/

defined function sequence

In [16]:
run(new SequenceTest(sequence))

cmd13$Helper$SequenceTest:
sequence
- should work


__b) (1 punto)__ Implementa la función `sequence` utilizando __`foldRight`__.

In [17]:
def sequenceFR[X, Y](list: List[Either[X, Y]]): Either[X, List[Y]] =
    list.foldRight(Right(List()): Either[X, List[Y]]){
        case (Left(x), _) =>
            Left(x)
        case (_, Left(x)) =>
            Left(x)
        case (Right(y), Right(ly)) =>
            Right(y :: ly)
    }

/*
    sequenceFR(List(Right(1), Left("error1"), Left("error2")))
                elem            acc
    1ª It. Left("error2")  Right(List()) => Left("error2")
    2ª It. Left("error1")  Left("error2") => Left("error1")
    3ª It. Right(1)        Left("error1") => Left("error1")
*/

defined function sequenceFR

In [18]:
run(new SequenceTest(sequenceFR))

cmd13$Helper$SequenceTest:
sequence
- should work


__c) (1 punto)__ Se desea implementar una función que reciba una lista de enteros, divida todos sus elementos entre un valor dado y, finalmente, multiplique los resultados de las divisiones, _siempre y cuando todas las divisiones hayan resultado en un valor entero_. En caso de que la división de algunos de los elementos no haya sido entera o no se haya podido realizar (en el caso de la división por cero), la función devolverá la cadena `"error"`. Por ejemplo:



In [20]:
class TestOp(op: (List[Int], Int) => Either[String, Int]) extends FlatSpec with Matchers {
    "op" should "work" in {
        op(List(2,4,6), 2) shouldBe Right(2/2*4/2*6/2)
        op(List(3,6,9), 3) shouldBe Right(6)
        op(List(3,5,9), 3) shouldBe Left("error")
        op(List(), 5) shouldBe Right(1)
        op(List(), 0) shouldBe Right(1)
        op(List(2,5,2), 0) shouldBe Left("error")
    }
}

defined class TestOp

Implementa la función `op` utilizando `sequence` y otras funciones de orden superior del catálogo explicado en clase.

In [19]:
def op(list: List[Int], n: Int): Either[String, Int] =
    sequence(list.map{ nList =>
        if (n != 0 && nList % n == 0) {
            Right(nList / n)
        } else {
            Left("error")
        }
    }) match {
        case Left(s) =>
            Left(s)
        case Right(l) =>
            Right(l.foldLeft(1){ (acc, elem) =>
                acc * elem
            })
    }

defined function op

In [21]:
run(new TestOp(op))

cmd19$Helper$TestOp:
op
- should work


# Ejercicio 4
__(3 puntos)__

Considérese la siguiente implementación mediante recursión final de la función de orden superior `foldLeft`: 
```scala
def foldLeft[A, B](l: List[A])(acc: B)(f: (B, A) => B): B = 
    l match {
        case Nil => acc
        case h :: t => foldLeft(t)(f(acc, h))(f)
    }
```

__a) (1,5 puntos)__ Implementa una variante de la función `foldLeft`, denominada `foldLeftUntil`, que permita finalizar de manera anticipada la iteración cuando el valor acumulado hasta el momento satisfaga determinada condición. En caso de que el valor acumulado no cumpla nunca la condición, el comportamiento será el mismo de la función `foldLeft`.

In [23]:
def foldLeftUntil[A, B](l: List[A])(acc: B)(pred: B => Boolean)(f: (B, A) => B): B =
    if (pred(acc)) {
        acc
    } else {
        l match {
            case List() =>
                acc
            case head :: tail =>
                foldLeftUntil(tail)(f(acc, head))(pred)(f)
        }
    }

defined function foldLeftUntil

_(variante 1)_

__b) (1,5 puntos)__ Dada una lista de pares clave/valor, se desea implementar una función `lookup` que devuelva el valor asociado a una clave en caso de que exista. Si existen varias ocurrencias con la misma clave se devolverá el valor de la primera. Por ejemplo:

In [24]:
class TestLookup(
    lookup: List[(Int, String)] => Int => Option[String]
) extends FlatSpec with Matchers {
    "lookup" should "work" in {
        lookup(List())(1) shouldBe None
        lookup(List((1, "a"), (2, "b"), (3, "c")))(1) shouldBe Some("a")
        lookup(List((1, "a"), (2, "b"), (1, "a_bis"), (3, "c")))(1) shouldBe Some("a")
        lookup(List((1, "a"), (2, "b"), (3, "c")))(2) shouldBe Some("b")
        lookup(List((1, "a"), (2, "b"), (3, "c")))(3) shouldBe Some("c")
        lookup(List((1, "a"), (2, "b"), (3, "c")))(0) shouldBe None
    }
}

defined class TestLookup

Implementa la función `lookup` utilizando la función `foldLeftUntil` del apartado anterior: 

In [27]:
// def foldLeftUntil[A, B](l: List[A])(acc: B)(pred: B => Boolean)(f: (B, A) => B): B
// isDefined: true -> Some | false -> None
def lookup[K, V](list: List[(K, V)])(key: K): Option[V] =
    foldLeftUntil(list)(None: Option[V])(_.isDefined){ // (acc, elem) =>
        /*
        case (acc, (k, v)) =>
            if (key == k) {
                Some(v)
            } else {
                acc
            }
        */
        case (_, (`key`, value)) => Some(value)
        case (acc, _) => acc
    }

defined function lookup

In [28]:
run(new TestLookup(lookup))

cmd23$Helper$TestLookup:
lookup
- should work


_(variante 2)_

__b) (1,5 puntos)__ Dada una lista de pares clave/valor, se desea implementar una función `lookup` que devuelva el valor asociado a una clave en caso de que exista. Si existen varias ocurrencias con la misma clave se devolverá el valor de la primera. Si no existe valor para la clave especificada se devolverá la cadena `"error"`. Por ejemplo:

In [30]:
class TestLookup(
    lookup: List[(Int, Char)] => Int => Either[String, Char]
) extends FlatSpec with Matchers {
    "lookup" should "work" in {
        lookup(List())(1) shouldBe Left("error")
        lookup(List((1, 'a'), (2, 'b'), (3, 'c')))(1) shouldBe Right('a')
        lookup(List((1, 'a'), (2, 'b'), (1, 'f'), (3, 'c')))(1) shouldBe Right('a')
        lookup(List((1, 'a'), (2, 'b'), (3, 'c')))(2) shouldBe Right('b')
        lookup(List((1, 'a'), (2, 'b'), (3, 'c')))(3) shouldBe Right('c')
        lookup(List((1, 'a'), (2, 'b'), (3, 'c')))(0) shouldBe Left("error")
    }
}

defined class TestLookup

Implementa la función `lookup` utilizando la función `foldLeftUntil` del apartado anterior: 

In [31]:
// isLeft: true Left | false Right
// isRight: true Right | false Left
def lookup[K, V](list: List[(K, V)])(key: K): Either[String, V] =
    foldLeftUntil(list)(Left("error"): Either[String, V])(_.isRight){ // (acc, elem) =>
        case (_, (`key`, value)) => Right(value)
        case (acc, _) => acc
    }

defined function lookup

In [32]:
run(new TestLookup(lookup))

cmd29$Helper$TestLookup:
lookup
- should work
